# Data Prep

In [ ]:
# prompt: download the pdf in content folder

import os

def download_pdf(pdf_url):
    try:
        os.system(f"wget -O /content/rag_data.pdf {pdf_url}")  # Download the PDF to the specified path
        print("PDF downloaded successfully!")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
pdf_url = "https://ggcity.org/pdf/pw/engineering/california%20building%20code%20section%20107%20to%20110.pdf"  # Replace with your PDF URL
download_pdf(pdf_url)

PDF downloaded successfully!


In [ ]:
# paul graham test

!mkdir -p '/content/data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O '/content/data/paul_graham/paul_graham_essay.txt'

--2024-11-06 17:49:04--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘/content/data/paul_graham/paul_graham_essay.txt’

/content/data/paul_ 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-11-06 17:49:04 (5.06 MB/s) - ‘/content/data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



# RAG with Openai

In [ ]:
%pip install llama-index-llms-openai --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [ ]:
!pip install llama-index --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.7 MB/s eta 0:00:00


In [ ]:
import logging
import sys
#level=logging.INFO
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import os

from getpass import getpass

# Define OpenAI API access
OPENAI_API_KEY = getpass('Enter the secret value: ')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

llm = OpenAI(model="gpt-4o-mini")

Settings.llm = llm

Settings.embed_model = OpenAIEmbedding()

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

index=VectorStoreIndex.from_documents(documents,show_progress=True)


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
from IPython.display import Markdown, display


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
display(Markdown(f"<b>{response}</b>"))



<b>Growing up, the author primarily worked on writing and programming outside of school. Initially, he wrote short stories, which he later described as lacking in plot and depth. His early programming experiences involved using an IBM 1401 at his school, where he learned to write programs in an early version of Fortran using punch cards. Eventually, he became more engaged with programming when he acquired a TRS-80 microcomputer, allowing him to write simple games and other programs. Despite his interest in programming, he initially planned to study philosophy in college, believing it to be a pursuit of ultimate truths, but later switched his focus to artificial intelligence.</b>

In [ ]:
#print(response.source_nodes[0])

for chunk in response.source_nodes:
  print(chunk)

Node ID: 0221054d-1504-47f8-a112-99146c81c2eb
Text: Now that I could write essays again, I wrote a bunch about
topics I'd had stacked up. I kept writing essays through 2020, but I
also started to think about other things I could work on. How should I
choose what to do? Well, how had I chosen what to work on in the past?
I wrote an essay for myself to answer that question, and I was
surprised how ...
Score:  0.818

Node ID: 210c8d9c-ec32-4116-ae3c-26e1078fbcab
Text: What I Worked On  February 2021  Before college the two main
things I worked on, outside of school, were writing and programming. I
didn't write essays. I wrote what beginning writers were supposed to
write then, and probably still are: short stories. My stories were
awful. They had hardly any plot, just characters with strong feelings,
which I ...
Score:  0.817



In [ ]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: Growing up, the author primarily worked on writing and
programming outside of school. Initially, he wrote short stories,
which he later described as lacking in plot and depth. His early
programming experiences involved using an IBM 1401 at his school,
where he learned to write programs in an early version of Fortran
using punch cards. Eventually, he became more engaged with programming
when he acquired a TRS-80 microcomputer, allowing him to write simple
games and other programs. Despite his interest in programming, he
initially planned to study philosophy in college, believing it to be a
pursuit of ultimate truths, but later switched his focus to artificial
intelligence.
______________________________________________________________________
Source Node 1/2
Node ID: 0221054d-1504-47f8-a112-99146c81c2eb
Similarity: 0.8176923406269162
Text: Now that I could write essays again, I wrote a bunch about
topics I'd had stacked up. I kept writing essays through 2020, but I
also 

## Storing Embedding

In [ ]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage2"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("Interesting info about Paul Graham")
print(response)

Response(
    response='Paul Graham has a diverse background in writing and programming, starting with his early experiences 
on an IBM 1401 in school, where he faced challenges due to the limitations of punch cards. His programming journey 
took off with the acquisition of a TRS-80 microcomputer, leading him to create simple games and a word processor. 
Initially intending to study philosophy in college, he found it unfulfilling and shifted his focus to artificial 
intelligence, inspired by literature and documentaries.\n\nGraham co-founded a company that developed a WYSIWYG 
site builder for online stores, which he described as a significant advancement in ecommerce software during the 
mid-1990s. His approach to pricing was influenced by his financial situation, leading to unexpectedly low 
subscription fees that attracted users. He emphasized the importance of high production values in making small 
online stores appear legitimate. Throughout his career, he has demonstrated a knack for innovation and adaptability
in the tech industry.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='88079e60-ef72-4f5b-acfe-a056bd39e12c',
                embedding=None,
                metadata={
                    'file_path': '/content/data/paul_graham/paul_graham_essay.txt',
                    'file_name': 'paul_graham_essay.txt',
                    'file_type': 'text/plain',
                    'file_size': 75042,
                    'creation_date': '2024-11-06',
                    'last_modified_date': '2024-11-06'
                },
                excluded_embed_metadata_keys=[
                    'file_name',
                    'file_type',
                    'file_size',
                    'creation_date',
                    'last_modified_date',
                    'last_accessed_date'
                ],
                excluded_llm_metadata_keys=[
                    'file_name',
                    'file_type',
                    'file_size',
                    'creation_date',
                    'last_modified_date',
                    'last_accessed_date'
                ],
                relationships={
                    <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                        node_id='5a9f6adc-1d9b-449e-aee3-6295f7236234',
                        node_type=<ObjectType.DOCUMENT: '4'>,
                        metadata={
                            'file_path': '/content/data/paul_graham/paul_graham_essay.txt',
                            'file_name': 'paul_graham_essay.txt',
                            'file_type': 'text/plain',
                            'file_size': 75042,
                            'creation_date': '2024-11-06',
                            'last_modified_date': '2024-11-06'
                        },
                        hash='d884d24da8b0bf0d4d0facabe60ecf7a463d7172ee28661b77b9c581a3dc030b'
                    ),
                    <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
                        node_id='94aaa9db-74a6-49f6-8b05-d09015b4caec',
                        node_type=<ObjectType.TEXT: '1'>,
                        metadata={},
                        hash='f40e9425884992580d3721823d174c7c797c43629bfe65e607b2f8fc01d26517'
                    )
                },
                text='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside 
of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to 
write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters
with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 
that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14.
The school district\'s 1401 happened to be in the basement of our junior high school, 

## Basic App

In [ ]:
!pip install gradio --q

In [ ]:
import gradio as gr
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader

# Initialize LlamaIndex components
def initialize_index():
    llm = OpenAI(model="gpt-4o-mini")
    Settings.llm = llm
    Settings.embed_model = OpenAIEmbedding()

    # Load documents and create index
    documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    return index

# Create query engine
index = initialize_index()
query_engine = index.as_query_engine()

def process_query(query):
    """
    Process the user query and return both the response and source citations
    """
    # Get response from query engine
    response = query_engine.query(query)

    # Format main response
    main_response = str(response)

    # Format citations
    citations = []
    for i, chunk in enumerate(response.source_nodes, 1):
        citation = f"Source {i}:\n{chunk.text}\n\nMetadata: {chunk.metadata}\n"
        citations.append(citation)

    citations_text = "\n\n".join(citations)

    return main_response, citations_text

# Create Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# LlamaIndex Query System")

    with gr.Row():
        query_input = gr.Textbox(
            label="Enter your query",
            placeholder="What would you like to know?",
            lines=2
        )

    with gr.Row():
        submit_btn = gr.Button("Submit Query")

    with gr.Row():
        with gr.Column():
            response_output = gr.Textbox(
                label="Response",
                lines=8,
                interactive=False
            )

        with gr.Column():
            citations_output = gr.Textbox(
                label="Source Citations",
                lines=8,
                interactive=False
            )

    # Handle submission
    submit_btn.click(
        fn=process_query,
        inputs=[query_input],
        outputs=[response_output, citations_output]
    )

    gr.Markdown("""
    ### Instructions:
    1. Enter your query in the text box above
    2. Click 'Submit Query' to get results
    3. View the main response and supporting citations below
    """)

# Launch the app
if __name__ == "__main__":
    app.launch(share=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/22 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4353204eebf715ec27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# chunking, parsing

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)
nodes[0]

In [ ]:
print(len(nodes))

18

In [ ]:
import nltk
from llama_index.core.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    # how many sentences on either side to capture
    window_size=3,
    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence",
)



In [ ]:
nodes = splitter.get_nodes_from_documents(documents)
nodes[0]

In [ ]:
print(node_parser)

SentenceWindowNodeParser(
    include_metadata=True,
    include_prev_next_rel=True,
    callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7a14aba412a0>,
    id_func=<function default_id_func at 0x7a14e23c05e0>,
    sentence_splitter=<function split_by_sentence_tokenizer.<locals>.<lambda> at 0x7a14836d7ac0>,
    window_size=3,
    window_metadata_key='window',
    original_text_metadata_key='original_sentence'
)

# RAG OS

In [ ]:
# load documents
documents = SimpleDirectoryReader("/content/data/paul_graham/").load_data()

In [ ]:
print(len(documents))

1

In [ ]:
#print(documents)

for doc in documents:
  print(doc.text)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings


embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

Settings.embed_model = embed_model
index2 = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [ ]:
print(index2)

<llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7a14aba43fa0>

In [ ]:
query_engine = index2.as_query_engine()

response = query_engine.query("PG birth")

In [ ]:
print(response)

In [ ]:
# per-index
# openai
embed_model = OpenAIEmbedding()
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [ ]:
print(index)

<llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7a15d80785b0>